Metrics/approaches:

- Confusion matrix.  
- FP/FN.
- ROC Curve.
   - Thinking about classification thresholds.

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/umbertomig/POLI30Dpublic/main/data/pfizer.csv")
df.head(3)

,drug,covid
0,vaccine,1
1,vaccine,1
2,vaccine,1


In [6]:
mod = smf.logit(data = df, formula = "covid ~ drug").fit()

Optimization terminated successfully.
         Current function value: 0.027286
         Iterations 12


Datasets:

- Pfizer (vaccine)
- Spam